In [1]:
# 8.1.3 Procedures
# The procedures can be found in the canvas module. Check the following under topics:
# 8.1 Weather Data Collection

## 8.1 Weather Data Collection

### Collecting weather data from an API

In [27]:
import requests
def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.
    
    Parameters:
    - endpoint: The endpoint of the API you want to
    make a GET request to.
    - payload: A dictionary of data to pass along
    with the request.
    
    Returns:
    Response object.
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
        'token': 'PASTE_YOUR_TOKEN_HERE'
        },
        params=payload
    )

### Collect All Data Points for 2018 In NYC (Various Stations)

In [35]:
import datetime
from IPython import display # for updating the cell dynamically
current = datetime.date(2018, 1, 1)
end = datetime.date(2019, 1, 1)
results = []
while current < end:
 # update the cell with status information
 display.clear_output(wait=True)
 display.display(f'Gathering data for {str(current)}')

 response = make_request(
 'data',
 {
 'datasetid' : 'GHCND', # Global Historical Climatology Network - Daily (GHCND) dataset
 'locationid' : 'CITY:US360019', # NYC
 'startdate' : current,
 'enddate' : current,
 'units' : 'metric',
 'limit' : 1000 # max allowed
 }
 )
 if response.ok:
     # we extend the list instead of appending to avoid getting a nested list
     results.extend(response.json()['results'])
 # update the current date to avoid an infinite loop
 current += datetime.timedelta(days=1)

'Gathering data for 2018-12-31'

In [33]:
import pandas as pd
df = pd.DataFrame(results)
df.head()

NameError: name 'results' is not defined

In [5]:
import pandas as pd
df.to_csv('datasets/nyc_weather_2018.csv', index=False)

NameError: name 'df' is not defined

In [23]:
import sqlite3
import pandas as pd

df = pd.read_csv('datasets/nyc_weather_2018.csv') # the data was taking too long that I do not know if there is something wrong, I added it manually instead...

with sqlite3.connect('databases/weather.db') as connection:
    df.to_sql(
        'weather', connection, index=False, if_exists='replace'
    )

In [29]:
response = make_request(
    'stations',
    {
        'datasetid' : 'GHCND', # Global Historical Climatology Network - Daily (GHCND) dataset
        'locationid' : 'CITY:US360019', # NYC
        'limit' : 1000 # max allowed
    }
)
stations = pd.DataFrame(response.json()['results'])[['id', 'name', 'latitude', 'longitude', 'elevation']]
stations.to_csv('data/weather_stations.csv', index=False)

with sqlite3.connect('data/weather.db') as connection:
     stations.to_sql(
         'stations', connection, index=False, if_exists='replace'
         )

KeyError: 'results'